# Evaluate Registration Results Applied to Label Annotations

## Overview

In this notebook we apply registration results and evaluate resulting label overlap measures.

## Inputs

1. Expert label segmentations in the input SmartSPIM image domain. Input labels may be supplied in one of two forms:
  1. As a single label volume with non-overlapping segmentations.
  2. As a series of binary label volumes assumed to each represent a single segmentation region. The series will be composited to create a single label volume.
  
2. Baseline label regions masked from the Allen Mouse Brain Common Coordinate Framework (CCF) v3.1 label image. Labels and regions of interest are assumed to correspond to SmartSPIM expert segmentations.

3D Slicer or a similar application may be used to create the _asymmetric_ baseline label image via the following steps:
1. Load the CCF v3.1 label atlas as an image volume.
2. Go to the "Segment Editor" module and create a new segmentation for the CCF image.
3. For each label of interest, use the "Binary Threshold" tool to segment the desired label.
4. For each label of interest, use the "Islands" tool to keep the island most closely matching the corresponding SmartSPIM segmentation by clicking on the island of interest. This step is intended to correct for mirroring that is present in the CCF atlas but not typically present in SmartSPIM annotations.
5. Create an additional segment. Use the "Logical Operators" tool to "add" all of the above segments into a single mask.
6. Use the "Mask Volume" tool to apply the mask from the previous step to the CCF volume. The result should be a label image that reflects CCF label values with asymmetric islands that roughly match the reference position of each expert annotation.

## Outputs

1. Dice coefficient metric value for evaluation of overlap agreement between CCF and aligned SmartSPIM label annotations. Dice coefficient is provided for the overall image and for each individual label region overlap.

2. (Optional) Composited SmartSPIM input label annotation image.


## Initialize Notebook

In [1]:
import os
import re

import itk
import itkwidgets
import numpy as np
import numpy.typing as npt
import pandas as pd

itk.auto_progress(1)

import sys

sys.path.append("../src")
import aind_ccf_alignment_experiments.registration_methods
from aind_ccf_alignment_experiments.registration_methods import (
    resample_label_image,
    get_sample_bounds,
)
from aind_ccf_alignment_experiments.label_methods import (
    compose_label_image,
    compute_dice_coefficient,
)

Loading ITKStatistics... Loading ITKStatistics... Loading ITKImageFilterBase... Loading ITKImageFilterBase... Loading ITKTransform... Loading ITKTransform... Loading ITKImageSources... Loading ITKImageSources... Loading ITKImageFunction... Loading ITKImageFunction... Loading ITKImageGrid... Loading ITKImageGrid... Loading ITKFFT... Loading ITKFFT... Loading ITKMesh... Loading ITKMesh... Loading ITKSpatialObjects... Loading ITKSpatialObjects... Loading ITKImageCompose... Loading ITKImageCompose... Loading ITKImageStatistics... Loading ITKImageStatistics... Loading ITKPath... Loading ITKPath... Loading ITKImageIntensity... Loading ITKImageIntensity... Loading ITKThresholding... Loading ITKThresholding... Loading ITKConvolution... Loading ITKConvolution... Loading ITKSmoothing... Loading ITKSmoothing... Loading ITKOptimizers... Loading ITKOptimizers... Loading ITKImageGradient... Loading ITKImageGradient... Loading ITKImageFeature... Loading ITKImageFeature... Loading ITKFiniteDifference.

In [3]:
# Reference for what labels to expect in the iamge
# and to optionally reconstruct label image from binary annotations
labels_info = pd.DataFrame(
    [
        ["VIIn", 788, "LSh_7n"],
        ["MH", 473, "LSh_MHN"],
        ["fr", 585, "LSh_Fr"],
        ["aco", 890, "LSh_main_part_of_aco"],
    ],
    columns=["ccf_name", "ccf_label_value", "seg_name"],
)

In [4]:
# Result of intensity registration. See `RegisterWithCCF.ipynb`
INPUT_TRANSFORM_FILEPATH = rf"D:\repos\allen-registration\notebooks\data\results\652506\BSPLINE\0.5mm_grid\2023.06.23\Ex_488_Em_525\652506_Ex_488_Em_525_transformresult.h5"

SUBJECT_ID = int(
    re.match(".*results\\\\([0-9]*).*", INPUT_TRANSFORM_FILEPATH).group(1)
)
CHANNEL_NAME = re.match(
    ".*(Ex_[0-9]*_Em_[0-9]*).*", INPUT_TRANSFORM_FILEPATH
).group(1)
SAMPLE_NAME = f"{SUBJECT_ID}_{CHANNEL_NAME}"

# See header instructions for constructing CCF baseline with 3D Slicer
TARGET_LABEL_IMAGE_FILEPATH = (
    rf"data\input\{SUBJECT_ID}\annotation\652506_CCF_Baseline.nii.gz"
)

# Expect either 1) composed label image has already been generated or
# 2) SmartSPIM label annotation image NIFTI files are present in the given directory
# with filenames matching the names provided in `labels_info` above
INPUT_LABEL_IMAGE_PATH = rf"data\input\{SUBJECT_ID}\annotation"
INPUT_LABEL_IMAGE_FILEPATH = (
    f"{INPUT_LABEL_IMAGE_PATH}/{SUBJECT_ID}_composed_labels.nii.gz"
)

RESULTS_PATH = f"{os.path.dirname(INPUT_TRANSFORM_FILEPATH)}/labels"
LABEL_IMAGE_RESULT_FILEPATH = rf"{RESULTS_PATH}\{SAMPLE_NAME}_labels.nii.gz"
OVERLAP_RESULT_FILEPATH = rf"{RESULTS_PATH}/{SAMPLE_NAME}_overlap.csv"

print(f"Write results to {RESULTS_PATH}")

Write results to D:\repos\allen-registration\notebooks\data\results\652506\BSPLINE\0.5mm_grid\2023.06.23\Ex_488_Em_525/labels


In [5]:
# set to True to view unique label values at each step
VERIFY_LABELS = False

## Load and Compose Label Image

Load the source label image composed of expert annotations. If a single composited label image does not already exist, create it by rescaling provided expert annotations and adding them together.

Also load the asymmetrically masked CCF atlas with corresponding label regions.

In [6]:
if os.path.exists(INPUT_LABEL_IMAGE_FILEPATH):
    source_label_image = itk.imread(
        INPUT_LABEL_IMAGE_FILEPATH, pixel_type=itk.SS
    )
else:
    source_label_image = compose_label_image(
        labels_info,
        input_labels_path=INPUT_LABEL_IMAGE_PATH,
        output_path=INPUT_LABEL_IMAGE_FILEPATH,
        verbose=True,
    )

Loading ITKPyUtils... Loading ITKPyUtils... itkImageFileReaderISS3: 0.000000itkImageFileReaderISS3: 1.000000

In [7]:
if VERIFY_LABELS:
    unique_vals = np.unique(source_label_image)
    print(f"Source label image has labels {unique_vals}")
    assert (
        len(np.unique(source_label_image)) == len(labels_info) + 1
    ), f"Expected {len(labels_info)} labels + 1 background value"

In [8]:
target_label_image = itk.imread(TARGET_LABEL_IMAGE_FILEPATH, pixel_type=itk.SS)
print(target_label_image)
if VERIFY_LABELS:
    print(np.unique(target_label_image))

itkImageFileReaderISS3: 0.000000itkImageFileReaderISS3: 1.000000

Image (0000022A458A1CA0)
  RTTI typeinfo:   class itk::Image<short,3>
  Reference Count: 2
  Modified Time: 879
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (0000022A41A01030) 
  Source output name: Primary
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 694
  UpdateMTime: 880
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [1320, 800, 1140]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [1320, 800, 1140]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [1320, 800, 1140]
  Spacing: [0.01, 0.01, 0.01]
  Origin: [5.695, -5.35, 5.22]
  Direction: 
0 0 -1
1 0 0
0 -1 0

  IndexToPointMatrix: 
0 0 -0.01
0.01 0 0
0 -0.01 0

  PointToIndexMatrix: 
0 100 0
0 0 -100
-100 0 0

  Inverse Direction: 
0 1 0
0 0 -1
-1 0 0

  PixelContainer: 
    ImportImageContainer (0000022A4E0063F0)
      RTTI typeinfo:   class itk::ImportImageContainer<unsigned __int6

## Load Registration Results

Get registration results from `RegisterToCCF.ipynb` to direct sampling from the target (CCF) domain to the source (SmartSPIM) domain. Verify that results approximately align with expectations by verifying that the transformed target centerpoint is near the transformed source centerpoint.

In [9]:
source_transform = itk.transformread(INPUT_TRANSFORM_FILEPATH)[0]

print(source_transform)

CompositeTransform (0000022A452E1990)
  RTTI typeinfo:   class itk::CompositeTransform<double,3>
  Reference Count: 2
  Modified Time: 576
  Debug: Off
  Object Name: 
  Observers: 
    none
  Transforms in queue, from begin to end:
  >>>>>>>>>
  TranslationTransform (0000022A5DD4FE60)
    RTTI typeinfo:   class itk::TranslationTransform<double,3>
    Reference Count: 2
    Modified Time: 514
    Debug: Off
    Object Name: 
    Observers: 
      none
    Offset: [6.6309, 7.9641, 2.4635]
  >>>>>>>>>
  BSplineTransform (0000022A4E3DD0D0)
    RTTI typeinfo:   class itk::BSplineTransform<double,3,3>
    Reference Count: 2
    Modified Time: 562
    Debug: Off
    Object Name: 
    Observers: 
      none
    CoefficientImage: [ 0000022A458A3B90, 0000022A458A27E0, 0000022A458A0BC0 ]
    TransformDomainOrigin: [6.89812, -6.83558, 6.11095]
    TransformDomainPhysicalDimensions: [16.0192, 10.4066, 13.1894]
    TransformDomainDirection: -0 0 -1
1 -0 0
0 -1 0

    TransformDomainMeshSize: [27, 1

Loading ITKIOTransformBase... Loading ITKIOTransformHDF5... Loading ITKIOTransformHDF5... Loading ITKIOTransformInsightLegacy... Loading ITKIOTransformInsightLegacy... Loading ITKIOTransformMatlab... Loading ITKIOTransformMatlab... Loading ITKIOTransformBase... 

In [10]:
def get_midpoint(image: itk.Image) -> npt.ArrayLike:
    bounds = get_sample_bounds(image)
    return (bounds[0] + bounds[1]) / 2


print(f"Target midpoint: {get_midpoint(target_label_image)}")

# Verify transformed target centerpoint is near source centerpoint
print(
    f"Transformed target midpoint: {np.array(source_transform.TransformPoint(get_midpoint(target_label_image)))}"
)
print(f"Source midpoint: {get_midpoint(source_label_image)}")

Target midpoint: [-0.0049997   1.24999995  1.21999988]
Transformed target midpoint: [6.9394849  9.16363091 3.25424755]
Source midpoint: [6.64560028 9.20160079 3.69599982]


## Map Expert Labels to CCF Space

Resample labels from SmartSPIM to CCF space with the given transform mapping. `itk.LabelImageGenericInterpolateImageFunction` is used to avoid artifacts from label interpolation.

In [11]:
resampled_source_label_image = resample_label_image(
    source_image=source_label_image,
    reference_image=target_label_image,
    transform=source_transform,
)

if VERIFY_LABELS:
    print(np.unique(resampled_source_label_image))

Loading GenericLabelInterpolator... Warning: Unknown parameter 'itk::LinearInterpolateImageFunction' in template 'itk::LabelImageGenericInterpolateImageFunction'


Loading GenericLabelInterpolator... itkResampleImageFilterISS3ISS3: 0.000000itkResampleImageFilterISS3ISS3: 0.061294itkResampleImageFilterISS3ISS3: 0.063048itkResampleImageFilterISS3ISS3: 0.116667itkResampleImageFilterISS3ISS3: 0.117544itkResampleImageFilterISS3ISS3: 0.160526itkResampleImageFilterISS3ISS3: 0.161403itkResampleImageFilterISS3ISS3: 0.201316itkResampleImageFilterISS3ISS3: 0.202193itkResampleImageFilterISS3ISS3: 0.237500itkResampleImageFilterISS3ISS3: 0.238377itkResampleImageFilterISS3ISS3: 0.279605itkResampleImageFilterISS3ISS3: 0.280482itkResampleImageFilterISS3ISS3: 0.321710itkResampleImageFilterISS3ISS3: 0.322588itkResampleImageFilterISS3ISS3: 0.366228itkResampleImageFilterISS3ISS3: 0.367105itkResampleImageFilterISS3ISS3: 0.445395itkResampleImageFilterISS3ISS3: 0.447149itkResampleImageFilterISS3ISS3: 0.497807itkResampleImageFilterISS3ISS3: 0.498684itkResampleImageFilterISS3ISS3: 0.539035itkResampleImageFilterISS3ISS3: 0.539912itkResampleImageFilterISS3ISS3: 0.581579itkR

itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000

itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000itkResampleImageFilterISS3ISS3: 1.000000

## Evaluate Overlap

Evaluate label overlap agreement between individual regions and for the overall image.

In [12]:
dice_score, label_measures_filter = compute_dice_coefficient(
    source_image=resampled_source_label_image, target_image=target_label_image
)

overlap_headers = ["ccf_label_name", "ccf_label_value", "dice_score"]
overlap_agreement_results = pd.DataFrame(
    [["all", -1, dice_score]], columns=overlap_headers
)

for index, row in labels_info.iterrows():
    result = pd.DataFrame(
        [
            [
                row["ccf_name"],
                row["ccf_label_value"],
                label_measures_filter.GetDiceCoefficient(
                    row["ccf_label_value"]
                ),
            ]
        ],
        columns=overlap_headers,
    )
    overlap_agreement_results = pd.concat(
        [overlap_agreement_results, result], ignore_index=True
    )

print(overlap_agreement_results)

itkLabelOverlapMeasuresImageFilterISS3: 0.000000itkLabelOverlapMeasuresImageFilterISS3: 0.000000itkLabelOverlapMeasuresImageFilterISS3: 1.000000

  ccf_label_name  ccf_label_value  dice_score
0            all               -1    0.265520
1           VIIn              788    0.219311
2             MH              473    0.691117
3             fr              585    0.472643
4            aco              890    0.103598


itkLabelOverlapMeasuresImageFilterISS3: 1.000000

## Save Results

In [13]:
os.makedirs(RESULTS_PATH, exist_ok=True)
itk.imwrite(
    resampled_source_label_image, LABEL_IMAGE_RESULT_FILEPATH, compression=True
)
print(f"Transformed label written to {LABEL_IMAGE_RESULT_FILEPATH}")

Transformed label written to D:\repos\allen-registration\notebooks\data\results\652506\BSPLINE\0.5mm_grid\2023.06.23\Ex_488_Em_525/labels\652506_Ex_488_Em_525_labels.nii.gz


itkImageFileWriterISS3: 0.000000itkImageFileWriterISS3: 1.000000

In [14]:
overlap_agreement_results.to_csv(OVERLAP_RESULT_FILEPATH)
print(f"Dice results written to {OVERLAP_RESULT_FILEPATH}")

Dice results written to D:\repos\allen-registration\notebooks\data\results\652506\BSPLINE\0.5mm_grid\2023.06.23\Ex_488_Em_525/labels/652506_Ex_488_Em_525_overlap.csv
